In [2]:
import pandas as pd
import xarray as xr
xr.set_options(keep_attrs=True,
               display_expand_data=False)

import plotly.express as px
import plotly.io as pio
# pio.renderers.default='png'

import yfinance as yf



Download stock prices from Yahoo Finance

In [ ]:
data = yf.download('msft aapl goog', period="10y")


In [ ]:
data

Build an xarray Dataset to store prices

In [ ]:
ds = data['Adj Close'].stack().rename_axis(['date', 'asset']).rename('price').to_xarray().to_dataset()
ds

Calculate returns from prices, store in Dataset

In [ ]:
ds['ret'] = ds.price.diff(dim='date') / ds.price
ds

Calculate time series of realized volatility, exponentially weighted

In [ ]:
def get_volatility_set(ret, halflifes):
    return xr.concat([ds.ret.rolling_exp({'date': h}, window_type='halflife').std() 
                       for h in halflifes],
                       dim=pd.Index(halflifes, name='vol_type'))

In [ ]:
halflifes = [21, 63, 126, 252, 512]

ds['vol'] = get_volatility_set(ds.ret, halflifes)
ds

Calculate time series of correlation, exponentially weighted

In [ ]:
def get_ewm_corr(ret, halflife):
    return (ret.to_pandas()
            .ewm(halflife=halflife)
            .corr()
            .stack()
            .rename_axis(['date', 'ticker', 'ticker_1'])
            .rename('corr')
            .to_xarray())

def get_correlation_set(ret, halflifes):
    return xr.concat([get_ewm_corr(ret, h) for h in halflifes],
                     dim=pd.Index(halflifes, name='corr_type'))

In [ ]:
ds['corr'] = get_correlation_set(ds.ret, halflifes)
ds

Select a particular correlation matrix

In [ ]:
ds.corr.sel(date='2023-11-29', corr_type=63).to_pandas()

Look at the correlation of a stock pair over time

In [ ]:
ds.corr.sel(ticker='AAPL', ticker_1='GOOG', date=slice('2015', None)).plot(x='date', hue='corr_type')

In [ ]:
def fig_format(height=500, width=1000, template='plotly_white'):
    return dict(height=height, width=width, template=template)
    

In [ ]:
df = ds.corr.sel(ticker='AAPL', ticker_1='GOOG').to_series().reset_index()
# px.line(df, **fig_format())

In [ ]:
df

In [ ]:
df = ds.corr.sel(ticker='AAPL', ticker_1='MSFT', corr_type=63).to_series().reset_index()
# bpx.line(df, x='date', y='corr')


In [ ]:
df = px.data.gapminder().query("country=='Canada'")
fig = px.line(df, x="year", y="lifeExp", title='Life expectancy in Canada')
fig.show()

In [ ]:
# fig.show(renderer='png')
# fig.show()

In [ ]:
# pio.renderers.default='png'